In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#data is unique by gvkey and fyear
data = pd.read_csv('fundamentals_annual.csv')
data.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,conm,curcd,...,txdbcl,txdi,txp,txt,uopi,costat,ggroup,sic,spcindcd,spcseccd
0,2080,11/30/2000,2000,INDL,C,D,STD,BSET,BASSETT FURNITURE INDS,USD,...,NaN,1.708,0.000,4.671,NaN,A,2520,2511,315.0,976.0
1,2080,11/30/2001,2001,INDL,C,D,STD,BSET,BASSETT FURNITURE INDS,USD,...,NaN,-0.824,0.000,-1.042,NaN,A,2520,2511,315.0,976.0
2,2080,11/30/2002,2002,INDL,C,D,STD,BSET,BASSETT FURNITURE INDS,USD,...,NaN,2.215,0.000,2.369,NaN,A,2520,2511,315.0,976.0
3,2080,11/30/2003,2003,INDL,C,D,STD,BSET,BASSETT FURNITURE INDS,USD,...,NaN,-1.154,1.530,0.462,NaN,A,2520,2511,315.0,976.0
4,2080,11/30/2004,2004,INDL,C,D,STD,BSET,BASSETT FURNITURE INDS,USD,...,NaN,1.875,0.682,2.641,NaN,A,2520,2511,315.0,976.0


In [3]:
fyear = pd.read_csv('income.csv')
fyear.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,curcd,ni,pi,txpd,costat,ggroup
0,2080,11/30/2000,2000,INDL,C,D,STD,USD,10.032,15.067,1.994,A,2520
1,2080,11/30/2001,2001,INDL,C,D,STD,USD,-2.642,-3.684,1.485,A,2520
2,2080,11/30/2002,2002,INDL,C,D,STD,USD,6.741,9.110,0.350,A,2520
3,2080,11/30/2003,2003,INDL,C,D,STD,USD,-0.470,4.867,-3.366,A,2520
4,2080,11/30/2004,2004,INDL,C,D,STD,USD,8.209,10.850,1.700,A,2520


In [4]:
data = data.merge(fyear, on = ['gvkey','fyear'],  suffixes=('', '_drop'))
data = data[[c for c in data.columns if not c.endswith('_drop')]]

# fyear and fyrc share same values
# fdate is the date when the data is finalized for the fiscal year. Just fiscal year alone will not tell us anything. 

In [5]:
data.drop(['consol','popsrc','indfmt'],axis=1,inplace=True) #same for all rows
data.drop(['dvpd','opiti','tii','uopi'], axis=1, inplace=True) #NaN values only
data.drop(['gld','gleps','glp'], axis=1, inplace=True) #more than 90% values are NaN

In [6]:
#drop columns with constant value 
for i in data.columns:
    if(len(data[i].unique()) == 1):
        print(f'Dropped: {i}')
        data.drop(i, axis=1,inplace=True)
data.head()

Dropped: datafmt
Dropped: curcd
Dropped: costat
Dropped: ggroup


,gvkey,datadate,fyear,tic,conm,fyr,fdate,acominc,act,ap,...,txdbcl,txdi,txp,txt,sic,spcindcd,spcseccd,ni,pi,txpd
0,2080,11/30/2000,2000,BSET,BASSETT FURNITURE INDS,11,NaN,NaN,136.875,20.310,...,NaN,1.708,0.000,4.671,2511,315.0,976.0,10.032,15.067,1.994
1,2080,11/30/2001,2001,BSET,BASSETT FURNITURE INDS,11,NaN,3.047,98.602,15.010,...,NaN,-0.824,0.000,-1.042,2511,315.0,976.0,-2.642,-3.684,1.485
2,2080,11/30/2002,2002,BSET,BASSETT FURNITURE INDS,11,NaN,1.551,102.966,17.738,...,NaN,2.215,0.000,2.369,2511,315.0,976.0,6.741,9.110,0.350
3,2080,11/30/2003,2003,BSET,BASSETT FURNITURE INDS,11,NaN,2.531,100.697,15.127,...,NaN,-1.154,1.530,0.462,2511,315.0,976.0,-0.470,4.867,-3.366
4,2080,11/30/2004,2004,BSET,BASSETT FURNITURE INDS,11,NaN,2.818,102.677,19.948,...,NaN,1.875,0.682,2.641,2511,315.0,976.0,8.209,10.850,1.700


In [7]:
#fdate or the final date is the date the data is finalized for that period. 
#what is datadate?
#FYEAR indicates the fiscal year that the company is in at the time of DATADATE.
#Compustat it is pretty simple to understand, DATADATE gives us the annual close of fiscal period, in the case of MSFT
#(June FY) this might be 6/30/2010, and the FYEAR is 2010


In [8]:
#we can work with datadate and fyear
#fdate will give the end of the fiscal year. 
#datadate will tell the date the data was recorded
#fyr and fyear together should match the month and year of datadate

In [9]:
#methods
#1. filter by gvkey to get records for all fyear and then run a time series regression with lag variables. 
#2. Train a model with all firms in the dataset.


DATADATE is COMPUSTAT’s approximation of the fiscal period-end date, but I have seen cases where this is off a 
few days from the company’s actual fiscal 
year-end date in EDGAR filings because DATADATE will be the last day of the month. APDEDATE is defined by 
COMPUSTAT as, “This item represents the actual date the company closes accounting for the period, which can 
be different from the last day of the month in the period.” Both of these dates may be thought of as the 
Balance Sheet date one would find in the actual corporate fillings.

In [10]:
#how to deal with missing values? what should be the threshold for dropping columns missing values? 
data

,gvkey,datadate,fyear,tic,conm,fyr,fdate,acominc,act,ap,...,txdbcl,txdi,txp,txt,sic,spcindcd,spcseccd,ni,pi,txpd
0,2080,11/30/2000,2000,BSET,BASSETT FURNITURE INDS,11,NaN,NaN,136.875,20.310,...,NaN,1.708,0.000,4.671,2511,315.0,976.0,10.032,15.067,1.994
1,2080,11/30/2001,2001,BSET,BASSETT FURNITURE INDS,11,NaN,3.047,98.602,15.010,...,NaN,-0.824,0.000,-1.042,2511,315.0,976.0,-2.642,-3.684,1.485
2,2080,11/30/2002,2002,BSET,BASSETT FURNITURE INDS,11,NaN,1.551,102.966,17.738,...,NaN,2.215,0.000,2.369,2511,315.0,976.0,6.741,9.110,0.350
3,2080,11/30/2003,2003,BSET,BASSETT FURNITURE INDS,11,NaN,2.531,100.697,15.127,...,NaN,-1.154,1.530,0.462,2511,315.0,976.0,-0.470,4.867,-3.366
4,2080,11/30/2004,2004,BSET,BASSETT FURNITURE INDS,11,NaN,2.818,102.677,19.948,...,NaN,1.875,0.682,2.641,2511,315.0,976.0,8.209,10.850,1.700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756,271841,12/31/2016,2016,GFASY,GAFISA SA,12,06/05/2017,0.000,1045.186,87.455,...,0.0,27.468,15.936,30.764,1520,NaN,NaN,-357.677,-154.291,3.296
2757,271841,12/31/2017,2017,GFASY,GAFISA SA,12,06/30/2018,0.000,523.210,77.026,...,0.0,-7.829,14.018,-6.974,1520,NaN,NaN,-256.591,-293.292,0.855
2758,271841,12/31/2018,2018,GFASY,GAFISA SA,12,12/05/2019,0.000,438.044,60.239,...,0.0,-6.483,14.795,-5.618,1520,NaN,NaN,-108.368,-114.438,0.865
2759,271841,12/31/2019,2019,GFASY,GAFISA SA,12,07/13/2020,0.000,450.252,52.981,...,0.0,-9.269,17.382,-8.776,1520,NaN,NaN,-6.478,-15.344,0.493


In [11]:
#use datadate that falls in the previous fiscal year to predict the earnings of the company in the next year. 

In [12]:
#why is there such a wide gap between the two dates?
(pd.to_datetime(data['fdate']) - pd.to_datetime(data['datadate'])).sort_values(ascending =False)

1806   1850 days
1757   1773 days
1472   1767 days
1817   1609 days
1776   1602 days
          ...   
2717         NaT
2718         NaT
2719         NaT
2720         NaT
2721         NaT
Length: 2761, dtype: timedelta64[ns]

In [13]:
data.sort_values(by=['gvkey','fyear','fyr'],inplace=True)

In [14]:
data.isna().sum()

gvkey         0
datadate      0
fyear         0
tic           0
conm          0
fyr           0
fdate       702
acominc     190
act         377
ap           54
at           54
ch           88
citotal     591
cshpri       80
dltt         59
dvt          80
ebit         57
ebitda       71
gp           57
icapt        54
invt         54
lt           55
opeps        80
revt         57
seq          54
teq         486
txdba       514
txdbca      664
txdbcl      599
txdi        121
txp          93
txt          57
sic           0
spcindcd    700
spcseccd    700
ni           57
pi           57
txpd        292
dtype: int64

In [15]:
df = data[['gvkey', 'datadate', 'fyear','fyr', 'acominc','act', 'ap', 'at', 'ch', 'citotal', 'cshpri', 'dltt', 'dvt', 'ebit','ebitda', 'gp', 'icapt', 'invt', 'lt', 'opeps', 'revt', 'seq','txdi', 'txp', 'txt', 'sic','ni','pi','txpd']]
#fill missing values using forward fill and backward fill and take average
#what this means is that the asset value in that year lied between the the asset value the year before and the one
#the year after
temp = df.groupby('gvkey').fillna(method='ffill')
temp = temp.fillna(0)
temp2 = df.groupby('gvkey').fillna(method='bfill')
temp2 = temp2.fillna(0)

cols = ['acominc','act', 'ap', 'at', 'ch', 'citotal', 'cshpri', 'dltt', 'dvt', 'ebit','ebitda', 'gp', 'icapt', 'invt', 'lt', 'opeps', 'revt', 'seq','txdi', 'txp', 'txt', 'sic','ni','pi','txpd']
for i in cols:
    df[i] = (temp[i] + temp2[i])//2
    
df.isna().sum()

/var/folders/y3/v6fyb_rs72bcn7ftc3lvwvmw0000gn/T/ipykernel_25792/3217801653.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i] = (temp[i] + temp2[i])//2


gvkey       0
datadate    0
fyear       0
fyr         0
acominc     0
act         0
ap          0
at          0
ch          0
citotal     0
cshpri      0
dltt        0
dvt         0
ebit        0
ebitda      0
gp          0
icapt       0
invt        0
lt          0
opeps       0
revt        0
seq         0
txdi        0
txp         0
txt         0
sic         0
ni          0
pi          0
txpd        0
dtype: int64

In [16]:
temp = df[['acominc', 'act', 'ap', 'at', 'ch','citotal', 'cshpri', 'dltt', 'dvt', 'ebit', 'ebitda', 'gp', 'icapt','invt', 'lt', 'opeps', 'revt', 'seq', 'txdi', 'txp', 'txt', 'sic','ni','pi','txpd']]

In [17]:
#very high correlations between features. its expected since they are mathematically derived from each other
temp.corr().style.background_gradient(cmap='coolwarm')

,acominc,act,ap,at,ch,citotal,cshpri,dltt,dvt,ebit,ebitda,gp,icapt,invt,lt,opeps,revt,seq,txdi,txp,txt,sic,ni,pi,txpd
acominc,1.000000,-0.660595,-0.719120,-0.670117,-0.619941,0.197542,-0.274578,-0.486201,-0.005569,-0.143061,-0.354437,-0.268185,-0.512905,-0.266654,-0.713790,-0.000267,-0.673004,-0.464532,-0.175738,-0.492217,-0.281236,-0.052688,0.155334,0.021159,-0.241400
act,-0.660595,1.000000,0.882736,0.941584,0.938767,0.351673,0.652767,0.746726,0.557399,0.672532,0.849986,0.777169,0.919453,0.570970,0.904267,-0.003495,0.961794,0.912984,0.039714,0.871615,0.643778,0.028795,0.357350,0.523532,0.748009
ap,-0.719120,0.882736,1.000000,0.929933,0.819021,0.296287,0.497633,0.631922,0.367234,0.546511,0.701871,0.548431,0.780833,0.494488,0.947192,0.010480,0.847965,0.760863,0.075959,0.804181,0.534912,0.024593,0.290352,0.438642,0.609752
at,-0.670117,0.941584,0.929933,1.000000,0.894162,0.391279,0.572369,0.754288,0.521249,0.684430,0.830334,0.663911,0.919035,0.629543,0.986049,0.013954,0.903717,0.906856,0.034229,0.891457,0.639785,-0.016291,0.403707,0.565223,0.749889
ch,-0.619941,0.938767,0.819021,0.894162,1.000000,0.371550,0.617517,0.775421,0.534241,0.664888,0.820089,0.764434,0.906733,0.635383,0.849807,0.045079,0.917450,0.887697,0.049905,0.807940,0.616196,-0.015089,0.385995,0.532443,0.709629
citotal,0.197542,0.351673,0.296287,0.391279,0.371550,1.000000,0.323607,0.310195,0.570787,0.749937,0.626497,0.480728,0.461981,0.406624,0.332522,0.097572,0.318899,0.504492,-0.270604,0.431947,0.375960,-0.071282,0.941328,0.891285,0.576919
cshpri,-0.274578,0.652767,0.497633,0.572369,0.617517,0.323607,1.000000,0.595617,0.573011,0.596583,0.676884,0.731292,0.671826,0.523525,0.508371,-0.022924,0.659525,0.659961,0.003104,0.554563,0.538460,-0.021049,0.316020,0.451009,0.637049
dltt,-0.486201,0.746726,0.631922,0.754288,0.775421,0.310195,0.595617,1.000000,0.612243,0.662172,0.774872,0.725733,0.866547,0.765843,0.710160,0.033526,0.776721,0.779170,-0.032308,0.653499,0.566501,-0.117527,0.338674,0.477097,0.694919
dvt,-0.005569,0.557399,0.367234,0.521249,0.534241,0.570787,0.573011,0.612243,1.000000,0.842593,0.810908,0.812923,0.679446,0.538755,0.436124,0.014423,0.538675,0.681820,-0.101773,0.533349,0.655059,-0.027037,0.621606,0.737221,0.808250
ebit,-0.143061,0.672532,0.546511,0.684430,0.664888,0.749937,0.596583,0.662172,0.842593,1.000000,0.949706,0.841684,0.793105,0.676942,0.606462,0.114384,0.676173,0.804989,-0.070265,0.679892,0.747652,-0.081187,0.798564,0.910861,0.884865


In [18]:
from sklearn.decomposition import PCA
pca = PCA()
dataset = pd.DataFrame()
transformed = pca.fit_transform(temp.drop('ni',axis=1))
for i in range(0, len(transformed[0])):
    dataset[i] = transformed[:,i]
    print(i, temp['ni'].corr(dataset[i]))
    

0 0.3990354606611766
1 -0.1189586475859518
2 0.5291868432975254
3 -0.1895316562941621
4 -0.17439302627998157
5 0.19796631524298167
6 -0.40723797450216487
7 -0.44968172671493745
8 0.0611292624086065
9 -0.08681827921835131
10 -0.11107960130192128
11 -0.04489628796234607
12 0.03513671811833358
13 0.0808389635890443
14 0.0004245231808643108
15 0.032842334722462514
16 0.17148185074762715
17 -0.05543907813209585
18 0.008390132918850827
19 0.0025522096839458754
20 -0.03620127153343575
21 -0.022368305150012587
22 0.0010845767187404565
23 0.0006011918645396039


In [39]:
pca = PCA(n_components = 3)
transformed = pca.fit_transform(temp.drop('ni',axis=1))

In [40]:
print(pca.explained_variance_ratio_) #97% of the variance explained by first 3 principal components. 

[0.92357055 0.04730212 0.01385517]


In [42]:
print(pca.singular_values_) #eigen values

[1189526.93976543  269202.7735291   145695.15826523]


In [43]:
dataset = pd.DataFrame({'Prin1': transformed[:, 0], 'Prin2': transformed[:, 1], 'Prin3': transformed[:, 2]})
print(dataset)

            Prin1        Prin2       Prin3
0    -5906.484803   958.351000  -42.643573
1    -6006.819820   998.955450  -49.066454
2    -6007.655826   977.085084  -57.663154
3    -6021.775848   979.575725  -64.778557
4    -5997.803502   986.549485  -50.303325
...           ...          ...         ...
2756 -4298.583743  1488.971211  333.519718
2757 -5288.675693  1436.409858   49.245693
2758 -5500.082946  1291.138139   -5.999447
2759 -5622.824256  1305.670640  138.573743
2760 -5475.247220  1223.915999  206.403309

[2761 rows x 3 columns]


In [44]:
temp.corrwith(dataset.Prin1)

acominc   -0.677239
act        0.965939
ap         0.924104
at         0.993791
ch         0.919811
citotal    0.386532
cshpri     0.611385
dltt       0.783393
dvt        0.546951
ebit       0.703202
ebitda     0.857372
gp         0.721390
icapt      0.941859
invt       0.662006
lt         0.969206
opeps      0.020021
revt       0.944854
seq        0.928322
txdi       0.038106
txp        0.892343
txt        0.661361
sic       -0.018980
ni         0.399035
pi         0.566840
txpd       0.770922
dtype: float64

In [45]:
temp.corrwith(dataset.Prin2)

acominc   -0.168868
act       -0.138164
ap         0.186494
at         0.102344
ch        -0.174127
citotal   -0.115988
cshpri    -0.358644
dltt      -0.231912
dvt       -0.366479
ebit      -0.288037
ebitda    -0.298008
gp        -0.584552
icapt     -0.245149
invt      -0.375546
lt         0.241307
opeps     -0.060070
revt      -0.283050
seq       -0.260751
txdi       0.018953
txp        0.023910
txt       -0.250457
sic        0.072546
ni        -0.118959
pi        -0.163794
txpd      -0.283755
dtype: float64

In [46]:
temp.corrwith(dataset.Prin3)

acominc    0.492335
act       -0.113539
ap        -0.178376
at         0.039525
ch        -0.031199
citotal    0.508668
cshpri     0.078427
dltt       0.203325
dvt        0.458799
ebit       0.438537
ebitda     0.265804
gp         0.197592
icapt      0.211047
invt       0.324774
lt        -0.031346
opeps      0.018307
revt      -0.158123
seq        0.224395
txdi      -0.170925
txp        0.043244
txt        0.210612
sic       -0.190738
ni         0.529187
pi         0.504566
txpd       0.334435
dtype: float64

In [47]:
data.columns

Index(['gvkey', 'datadate', 'fyear', 'tic', 'conm', 'fyr', 'fdate', 'acominc',
       'act', 'ap', 'at', 'ch', 'citotal', 'cshpri', 'dltt', 'dvt', 'ebit',
       'ebitda', 'gp', 'icapt', 'invt', 'lt', 'opeps', 'revt', 'seq', 'teq',
       'txdba', 'txdbca', 'txdbcl', 'txdi', 'txp', 'txt', 'sic', 'spcindcd',
       'spcseccd', 'ni', 'pi', 'txpd'],
      dtype='object')